# Import modules (run once)

In [4]:
%matplotlib
import sys
import numpy as np
import visa
import matplotlib.pyplot as plt

# Add path
sys.path.append('..\Instrument Driver')
sys.path.append('..\Helper')

# import custom modules
import myplots_py3

# import instrument driver
import nidaqmx.stream_readers
# import AFG3252
import Lakeshore332

Using matplotlib backend: Qt5Agg


# Test

In [ ]:
%matplotlib
import time

sample_num = 2000
sample_rate = 10000  # 10k is max when two channels are used

data = np.zeros((2,sample_num), dtype=np.float64)

task = nidaqmx.task.Task() # create a task obj in NI-max
task.ai_channels.add_ai_voltage_chan("Dev1/ai0") # for I 
task.ai_channels.add_ai_voltage_chan("Dev1/ai1") # for V

task.triggers.start_trigger.cfg_dig_edge_start_trig('/Dev1/PFI0')
task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=sample_num) # set sampling rate per channel
# timingCfg = nidaqmx.task.Timing(task._handle) 
# timingCfg.cfg_samp_clk_timing(sample_rate, samps_per_chan=sample_num) # set sampling rate per channel

ai = nidaqmx.stream_readers.AnalogMultiChannelReader(task.in_stream)

DAQplotter = myplots_py3.plotDAQ(xlabel1='time (s)', ylabel1='Raw V for I (V)',
                              xlabel2='time (s)', ylabel2='Raw V for V (V)',
                              xlabel3='I (A)', ylabel3='V (V)',
                              title='VI Meas', figsize=(10,10))

# Take data continuously and plot it
indexArr = np.arange(sample_num)
#timeArr = np.arange(sample_num) * (1/sample_rate)

for i in range(1000):
    ai.read_many_sample(data, sample_num)
    time.sleep(1)
    Iarr, Varr = data[0,:], data[1,:]
    
    Varr = Varr  #+ np.random.rand(sample_num) * 0.1
    
    # data post-processing
    z = np.polyfit(Iarr, Varr, 1) # do linear fit
    Varr_fit = z[1] + z[0] * Iarr

    # plot VI data and linear fit
    DAQplotter.update(indexArr, Iarr, Varr, Varr_fit)  

task.close()

Using matplotlib backend: Qt5Agg


C:\Users\Gunnar\Anaconda3\lib\site-packages\nidaqmx\task.py:105: ResourceWarning: Task of name "_unnamedTask<2>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  'reserved.'.format(self.name), DaqResourceWarning)


# Class definition

In [2]:
# class DAQ(object):
#     """
#     DAQ factory with 2 analog inputs
#     Inputs: sample_num, sample_rate
#     """
#     def __init__(self, sample_num=sample_num, sample_rate=sample_rate):
#         self.sample_num = sample_num
#         self.sample_rate = sample_rate
#         self.data = np.array([,])
        
#         # create a task
#         self.task = nidaqmx.task.Task() 
#         self.ai = nidaqmx.stream_readers.AnalogMultiChannelReader(self.task.in_stream) # create ai
        
#         # set analog inputs
#         self.task.ai_channels.add_ai_voltage_chan("Dev1/ai0") # for I 
#         self.task.ai_channels.add_ai_voltage_chan("Dev1/ai1") # for V
        
#         # set external trigger
#         self.task.triggers.start_trigger.cfg_dig_edge_start_trig('/Dev1/PFI0')

#         # set sampling rate and number of samples
#         self.task.timing.cfg_samp_clk_timing(self.sample_rate, samps_per_chan=self.sample_num) # set sampling rate per channel
                      
#     def getData(self):        
#         self.ai.read_many_sample(self.data, self.sample_num)
#         return self.data


class VI_Meas(object):
    """
    Do VI measurement
    """
        
    def __init__(self, standard_resistance, preamp_gain_for_I, preamp_gain_for_V, sample_num, sample_rate ):
        self.standard_resistance = standard_resistance
        self.preamp_gain_for_I = preamp_gain_for_I
        self.preamp_gain_for_V = preamp_gain_for_V
        self.sample_num = sample_num
        self.sample_rate = sample_rate
        
        self._configure_DAQ()
        
        self.data= np.arrange(sample_num)
    
    def _configure_DAQ(self):
        # create a task
        self.task = nidaqmx.task.Task() 
        self.ai = nidaqmx.stream_readers.AnalogMultiChannelReader(self.task.in_stream) # create ai
        
        # set analog inputs
        self.task.ai_channels.add_ai_voltage_chan("Dev1/ai0") # for I 
        self.task.ai_channels.add_ai_voltage_chan("Dev1/ai1") # for V
        
        # set external trigger
        self.task.triggers.start_trigger.cfg_dig_edge_start_trig('/Dev1/PFI0')

        # set sampling rate and number of samples
        self.task.timing.cfg_samp_clk_timing(self.sample_rate, samps_per_chan=self.sample_num) # set sampling rate per channel
    
    def getVI(self):
        self.ai.read_many_sample(self.data, self.sample_num) # take data
        I = self.data[0,:] / self.preamp_gain_for_I / self.standard_resistance,  # convert preamp voltage to actual V and I
        V = self.data[1,:] / self.preamp_gain_for_V
       
        return V, I
    

# VI Meas with VI_meas class

In [ ]:
# Set parameters
standard_resistance = 10e3    # Ohm, standard resistor
preamp_gain_for_I = 1     # Preamp gain for I
preamp_gain_for_V = 1e3   # Preamp gain for V
sample_num = 2000
sample_rate = 20e3

# create VI_meas object
VI = VI_meas(standard_resistance, preamp_gain_for_I, preamp_gain_for_V
             , sample_num, sample_rate)

# Create a plot object
DAQplotter = VI_Plotter(xlabel1='time (s)', ylabel1='Raw V for I (V)'
                        , xlabel2='time (s)', ylabel2='Raw V for V (V)'
                        , title='VI Meas', figuresize=(12,8))

# Take data continuously and plot it
timedata = np.arange(0, sample_num / sample_rate, 1/sample_rate)  # CHECK THIS

for _ in range(10):
    # acquire data in DAQ
    V, I = VI.getVI()
    
    # data post-processing
    z = np.polyfit(I, V, 1) # do linear fit
    V_fit = z[1] + z[0] * I
  
    # plot VI data and linear fit
    DAQplotter.update(timedata, I, timedata, V, I, V, V, V_fit)  
        

# VI Meas. with DCoffset + AC (old)

In [ ]:
# Set parameters
R_std = 10e3    # Ohm, standard resistor
gainForI = 1     # Preamp gain for I
gainForV = 1e3   # Preamp gain for V

sampling_rate =  20e3 # Sampling rate of DAQ
num_sample = 2000     # number of samples to take

Vdc_start,Vdc_end = 0, 1 # V
sweepSeq = np.linspace(0, 1, 100)

# Create a plot object
DAQplotter = myplot_py3.plotDAQ(xlabel1='time (s)', ylabel1='Raw V for I (V)',
                              xlabel2='time (s)', ylabel2='Raw V for V (V)',
                              title='VI Meas', figuresize=(12,8))
mainPlotter = myplot_py3.plot1D(xlabel1='Current (A)', ylabel1='Voltage (V)', title='VI Meas', plotstyle='b-')

# initialize array
Iarr_avg, Varr_avg = np.array([]), np.array([])

for volt in sweepSeq:
    VSrc.setVoltage(volt)    # set DC offset
    
    with nidaqmx.stream_readers.AnalogMultiChannerlReader as ai:
            # acquire data in DAQ
            ai.read_many_sample(data, num_sample)   # take voltages from preamps
            Iarr, Varr = data[0,:] / gainForI / R_std, data[1,:] / gainForV # convert preamp voltage to actual V and I
            I, V = np.mean(Iarr), np.mean(Varr)
            
            # data post-processing
            z = np.polyfit(Iarra, Varr, 1) # do linear fit
            Varr_fit = z[1] + z[0] * Iarr
            
    Iarr_avg = np.append(Iarr_avg, I)
    Varr_avg = np.append(Varr_avg, V)
    
    DAQplotter.update(Iarr, Varr, Varr_fit)
    mainPlotter.update(Iarr_avg, Varr_avg)
    


# RT Meas

In [ ]:
# Set parameters
standard_resistance = 10e3    # Ohm, standard resistor
preamp_gain_for_I = 1     # Preamp gain for I
preamp_gain_for_V = 1e3   # Preamp gain for V
sample_num = 2000
sample_rate = 20e3

LS = Lakeshore332.Lakeshore332()

# create VI_meas object
VI = VI_meas(standard_resistance, preamp_gain_for_I, preamp_gain_for_V, sample_num, sample_rate)

# Create a plot object
DAQplotter = RT_Plotter(xlabel1='time (s)', ylabel1='Raw V for I (V)',
                              xlabel2='time (s)', ylabel2='Raw V for V (V)',
                              title='VI Meas', figuresize=(12,8))

# Take data continuously and plot it
timedata = np.arange(0, sample_num / sample_rate, 1/sample_rate)  # CHECK THIS

while True:
    # read temperature
    MXC_temperature = LS.getTemperature('B')
    
    # acquire data in DAQ
    V, I = VI.getVI()
    
    # data post-processing
    z = np.polyfit(I, V, 1) # do linear fit
    V_fit = z[1] + z[0] * I
  
    # plot VI data and linear fit
    DAQplotter.update(timedata, I, timedata, V, I, V, V, V_fit, MXC_temperature, Z[0])  
        

# Playground

In [23]:
np.arange(10)*1e-6

array([  0.00000000e+00,   1.00000000e-06,   2.00000000e-06,
         3.00000000e-06,   4.00000000e-06,   5.00000000e-06,
         6.00000000e-06,   7.00000000e-06,   8.00000000e-06,
         9.00000000e-06])

In [35]:
np.random.rand(10)

array([ 0.56152969,  0.44506761,  0.40571925,  0.77337491,  0.69283947,
        0.82859652,  0.92888809,  0.11322308,  0.23233362,  0.59160799])